In [1]:
from gssl_func import *
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.neighbors import kneighbors_graph
from random import sample
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from node2vec import Node2Vec
from sklearn.datasets import load_svmlight_file
from sklearn.svm import SVC
import scipy.io as sio



unable to import 'smart_open.gcs', disabling that module


In [13]:
digits = [8,9]

x, labels = mnist_dataset(str(digits[0])+'and'+str(digits[1]))
labels[np.where(labels==digits[0]+1)] = -1
labels[np.where(labels==digits[1]+1)] = 1
total_ind = np.array([i for i in range(labels.shape[0])])
# print(labels)


knn_w = np.array(kneighbors_graph(x, 10, mode='distance').todense())

for i in range(knn_w.shape[0]):
    for j in range(i+1, knn_w.shape[0]):
        knn_w[i][j] = max(knn_w[i][j], knn_w[j][i])
        knn_w[j][i] = max(knn_w[i][j], knn_w[j][i])

np.fill_diagonal(knn_w, 0)

In [ ]:
write_edgelist(knn_w, 'digitpair')
acc_lists = [[] for i in range(8)]
abs_lists = [[] for i in range(8)]
it_lists = [[] for i in range(8)]
for i in range(5):
    train_index = sample(list(np.where(labels==-1)[0]), 2) + sample(list(np.where(labels==1)[0]), 2)
    test_index = np.array(list(set(total_ind)-set(train_index)))
    train_index = np.array(train_index)
    print(train_index.shape[0], test_index.shape[0])
    clf = SVC()
    clf.fit(x[train_index], labels[train_index])
    acc_lists[5].append(clf.score(x[test_index], labels[test_index]))
    abs_lists[5].append(np.average(np.absolute(clf.predict(x[test_index]))))
    cla_acc, reg_err, y_pred = spielman_voltage(labels, train_index, test_index, 'digitpair', i, reg=False)
    acc_lists[0].append(cla_acc)
    abs_lists[0].append(np.average(np.absolute(y_pred[test_index])))
    cla_acc, reg_err, y_pred, it = johnsen(knn_w, labels, train_index, test_index, max_it=1000, reg=False)
    it_lists[1].append(it)
    acc_lists[1].append(cla_acc)
    abs_lists[1].append(np.average(np.absolute(y_pred[test_index])))
    cla_acc, reg_err, y_pred, it = label_prop(knn_w, labels, train_index, test_index, max_it=1000, reg=False)
    it_lists[2].append(it)
    acc_lists[2].append(cla_acc)
    abs_lists[2].append(np.average(np.absolute(y_pred[test_index])))
    acc, reg_err, y_pred, it  = iterate_hebbian(knn_w, labels, train_index, test_index, max_it = 1000, eta=0.2, reg=False)
    it_lists[3].append(it)
    acc_lists[3].append(acc)
    abs_lists[3].append(np.average(np.absolute(y_pred[test_index])))
    acc, reg_err, y_pred, it  = biased_assimilation(knn_w, labels, train_index, test_index, reg=False)
    it_lists[4].append(it)
    acc_lists[4].append(acc)
    abs_lists[4].append(np.average(np.absolute(y_pred[test_index])))
    acc, reg_err, y_pred, it  = edit_biased_assimilation(knn_w, labels, train_index, test_index, max_it = 1000, eta=0.2, reg=False)
    acc_lists[6].append(acc)
    it_lists[6].append(it)
    abs_lists[6].append(np.average(np.absolute(y_pred[test_index])))

In [16]:
left = 0
right = 5
print('method name\t\t average accuracy\t average abs value')
print('spielmans voltage:\t', np.average(acc_lists[0][left:right]), np.average(abs_lists[0][left:right]), np.average(it_lists[0][left:right]))
print('Friedkin-johnsen:\t', np.average(acc_lists[1][left:right]), np.average(abs_lists[1][left:right]), np.average(it_lists[1][left:right]))
print('Biased Assimilation:\t', np.average(acc_lists[4][left:right]), np.average(abs_lists[4][left:right]), np.average(it_lists[4][left:right]))
print('Label Propagation:\t', np.average(acc_lists[2][left:right]), np.average(abs_lists[2][left:right]), np.average(it_lists[2][left:right]))
print('Iterative Hebbian:\t', np.average(acc_lists[3][left:right]), np.average(abs_lists[3][left:right]), np.average(it_lists[3][left:right]))
print('Hebbian + Biased:\t', np.average(acc_lists[6][left:right]), np.average(abs_lists[6][left:right]), np.average(it_lists[6][left:right]))
#     print('Iterative Hebbian4:\t', np.average(acc_lists[7][left:right]), np.average(abs_lists[7][left:right]), np.average(it_lists[7][left:right]))
print('SVM:\t\t\t', np.average(acc_lists[5][left:right]))

method name		 average accuracy	 average abs value
spielmans voltage:	 0.9507348029392118 0.29405562907244515 nan
Friedkin-johnsen:	 0.9777889111556446 0.0014168764215407744 27.0
Biased Assimilation:	 0.9689378757515029 0.9999733592609732 65.33333333333333
Label Propagation:	 0.7832331329325317 0.10619726918975758 1000.0
Iterative Hebbian:	 0.9741148964595858 0.9443287721156465 1000.0
Hebbian + Biased:	 0.96125584502338 0.9999925330455101 47.666666666666664
SVM:			 0.8124582498329994


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/numpy/lib/function_base.py:390: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
